In [1]:
from itertools import islice


def to_batches(it, n):
    it = iter(it)
    return iter(lambda: tuple(islice(it, n)), ())

In [ ]:
import torch
from meshae.model import FeatureEmbeddingConfig, MeshAEEmbedding


cfg = {
    "area": FeatureEmbeddingConfig(),
    "norm": FeatureEmbeddingConfig(),
    "angle": FeatureEmbeddingConfig(),
    "vertex": FeatureEmbeddingConfig(),
}
emb = MeshAEEmbedding(cfg)

In [4]:
emb

MeshAEEmbedding(
  (embeddings): ModuleDict(
    (area): Embedding(129, 128, padding_idx=0)
    (norm): Embedding(129, 128, padding_idx=0)
    (angle): Embedding(129, 128, padding_idx=0)
    (vertex): Embedding(129, 128, padding_idx=0)
  )
  (proj_in): Sequential(
    (0): Linear(in_features=2048, out_features=512, bias=True)
    (1): GELU(approximate='none')
  )
  (sageconv_in): SAGEConv(512, 512, aggr=mean)
  (sageconv_activate): Sequential(
    (0): GELU(approximate='none')
    (1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (sageconv_hidden): ModuleList()
)